# Assignment 4
## Question 2 - Workforce Planning

In [110]:
from pulp import *
import pandas as pd

In [111]:
prob2 = LpProblem('Prob2', LpMinimize)
month = [1, 2, 3, 4, 5, 6]

Initial = 145
Demand_per = [100,100,115,125,140,150] 

# Create a dictionary for Demand_per_mon
Demand_per_mon= makeDict([month], Demand_per) 
print(Demand_per_mon)

{1: 100, 2: 100, 3: 115, 4: 125, 5: 140, 6: 150}


In [112]:
# ============================
# Define Decision Variables
# ============================
Trained = LpVariable.dicts('Training Decision', month,lowBound = 0, cat = 'Continuous')
Layoff = LpVariable.dicts('Layoff Decision', month, lowBound = 0, cat = 'Continuous')
Num =  LpVariable.dicts('Number of Employees for each month', month, lowBound = 0, cat = 'Continuous')
print(Trained)
print(Layoff)
print(Num)

{1: Training_Decision_1, 2: Training_Decision_2, 3: Training_Decision_3, 4: Training_Decision_4, 5: Training_Decision_5, 6: Training_Decision_6}
{1: Layoff_Decision_1, 2: Layoff_Decision_2, 3: Layoff_Decision_3, 4: Layoff_Decision_4, 5: Layoff_Decision_5, 6: Layoff_Decision_6}
{1: Number_of_Employees_for_each_month_1, 2: Number_of_Employees_for_each_month_2, 3: Number_of_Employees_for_each_month_3, 4: Number_of_Employees_for_each_month_4, 5: Number_of_Employees_for_each_month_5, 6: Number_of_Employees_for_each_month_6}


In [113]:
# ===========================
# Define Objective Function
# ===========================
prob2 += lpSum([(6-i)*3300*0.9*Trained[i] for i in month[0:5]])\
      + 3000*Num[1]\
      + lpSum([3000*(Num[i] - sum(0.9*Trained[j] for j in month[0:i-1])) for i in month[1:6]])
prob2

Prob2:
MINIMIZE
3000*Number_of_Employees_for_each_month_1 + 3000*Number_of_Employees_for_each_month_2 + 3000*Number_of_Employees_for_each_month_3 + 3000*Number_of_Employees_for_each_month_4 + 3000*Number_of_Employees_for_each_month_5 + 3000*Number_of_Employees_for_each_month_6 + 1350.0*Training_Decision_1 + 1080.0*Training_Decision_2 + 810.0*Training_Decision_3 + 540.0*Training_Decision_4 + 270.0*Training_Decision_5 + 0.0
VARIABLES
Number_of_Employees_for_each_month_1 Continuous
Number_of_Employees_for_each_month_2 Continuous
Number_of_Employees_for_each_month_3 Continuous
Number_of_Employees_for_each_month_4 Continuous
Number_of_Employees_for_each_month_5 Continuous
Number_of_Employees_for_each_month_6 Continuous
Training_Decision_1 Continuous
Training_Decision_2 Continuous
Training_Decision_3 Continuous
Training_Decision_4 Continuous
Training_Decision_5 Continuous

In [114]:
# ===============================================================================
# Define Constraints: Labor Demand, Monthly Employment Limits, Training limits
# ===============================================================================
prob2 += Num[1] == 145
prob2 += Num[1] - Trained[1] >= 100,'Labor demand month 1'
prob2 += Num[6] + 0.2*0.9*(sum(Trained[j] for j in month[0:5])) >= Demand_per_mon[6],'Labor demand month 6'
for i in month[1:5]:
    prob2 += Num[i] - Trained[i] + 0.2*0.9*(sum(Trained[j] for j in month[0:i-1])) >= Demand_per_mon[i],'Labor demand month %s'%i

# Next month’s total employment = reduce the layoff amount 
for i in month[0:5]:
    prob2 += Num[i] - Layoff[i] == Num[i+1],'Inflow-outflow balance %s'%i
    
# Each month‘s layoff = 5% of (total employees - trained programmers - programmers on training)
prob2 += Layoff[1] == 0.05*(Num[1] - Trained[1]),'Layoff month 1'
for i in month[1:5]:
     prob2 += Layoff[i] == 0.05*(Num[i] - sum(Trained[j] for j in month[0:i])),'Layoff month %s'%i
prob2 += Layoff[6] == 0.05*(Num[6] - Trained[1] - Trained[2] - Trained[3] - Trained[4] - Trained[5]),'Layoff month 6'

    
# Each month’s # of trained programmers <= # of untrained programmers
for i in month[1:5]:
      prob2 += Trained[i] <= Num[i] - sum(Trained[j] for j in month[0:i-1]),'Train limit month %s'%i
prob2 += Trained[1] <= Num[1],'Train limit month 1'
prob2

Prob2:
MINIMIZE
3000*Number_of_Employees_for_each_month_1 + 3000*Number_of_Employees_for_each_month_2 + 3000*Number_of_Employees_for_each_month_3 + 3000*Number_of_Employees_for_each_month_4 + 3000*Number_of_Employees_for_each_month_5 + 3000*Number_of_Employees_for_each_month_6 + 1350.0*Training_Decision_1 + 1080.0*Training_Decision_2 + 810.0*Training_Decision_3 + 540.0*Training_Decision_4 + 270.0*Training_Decision_5 + 0.0
SUBJECT TO
_C1: Number_of_Employees_for_each_month_1 = 145

Labor_demand_month_1: Number_of_Employees_for_each_month_1
 - Training_Decision_1 >= 100

Labor_demand_month_6: Number_of_Employees_for_each_month_6
 + 0.18 Training_Decision_1 + 0.18 Training_Decision_2
 + 0.18 Training_Decision_3 + 0.18 Training_Decision_4
 + 0.18 Training_Decision_5 >= 150

Labor_demand_month_2: Number_of_Employees_for_each_month_2
 + 0.18 Training_Decision_1 - Training_Decision_2 >= 100

Labor_demand_month_3: Number_of_Employees_for_each_month_3
 + 0.18 Training_Decision_1 + 0.18 Training

In [115]:
# =========================
# Solving the PULP model
# =========================
prob2.solve()
print("Status:",LpStatus[prob2.status])

Status: Optimal


In [116]:
# =======================================================
# Print Out the Optimal Solution and Decision Variables
# =======================================================
for i in prob2.variables():
    print(i.name, "=", i.varValue,"\tReduced Cost =", i.dj)
print("The minimize total operating cost for 6 months is ", value(prob2.objective))

Layoff_Decision_1 = 6.5817658 	Reduced Cost = 0.0
Layoff_Decision_2 = 4.2114837 	Reduced Cost = 4.5474735e-13
Layoff_Decision_3 = 2.5528749 	Reduced Cost = 2.2737368e-13
Layoff_Decision_4 = 1.3441942 	Reduced Cost = 0.0
Layoff_Decision_5 = 0.81857044 	Reduced Cost = 4.5474735e-13
Layoff_Decision_6 = 0.77764192 	Reduced Cost = 0.0
Number_of_Employees_for_each_month_1 = 145.0 	Reduced Cost = -1.080025e-12
Number_of_Employees_for_each_month_2 = 138.41823 	Reduced Cost = 4.5474735e-13
Number_of_Employees_for_each_month_3 = 134.20675 	Reduced Cost = 9.9475983e-14
Number_of_Employees_for_each_month_4 = 131.65388 	Reduced Cost = 3.6948222e-13
Number_of_Employees_for_each_month_5 = 130.30968 	Reduced Cost = -1.4210855e-13
Number_of_Employees_for_each_month_6 = 129.49111 	Reduced Cost = -9.094947e-13
Training_Decision_1 = 13.364683 	Reduced Cost = -3.9790393e-13
Training_Decision_2 = 40.823877 	Reduced Cost = 5.6843419e-14
Training_Decision_3 = 28.960691 	Reduced Cost = -8.5265128e-13
Training_

In [117]:
print("\nSensitivity Analysis\nConstraint\t\t\t\t\t\t\tShadow Price\t\tSlack")
for name,c in list(prob2.constraints.items()):
    print(name,":",c,"\t",c.pi,"\t\t",c.slack)


Sensitivity Analysis
Constraint							Shadow Price		Slack
_C1 : Number_of_Employees_for_each_month_1 = 145 	 8440.1369 		 -0.0
Labor_demand_month_1 : Number_of_Employees_for_each_month_1 - Training_Decision_1 >= 100 	 0.0 		 -31.635320000000007
Labor_demand_month_6 : Number_of_Employees_for_each_month_6 + 0.18000000000000002*Training_Decision_1 + 0.18000000000000002*Training_Decision_2 + 0.18000000000000002*Training_Decision_3 + 0.18000000000000002*Training_Decision_4 + 0.18000000000000002*Training_Decision_5 >= 150.0 	 6089.6523 		 -0.0
Labor_demand_month_2 : Number_of_Employees_for_each_month_2 + 0.18000000000000002*Training_Decision_1 - Training_Decision_2 >= 100.0 	 471.46017 		 -0.0
Labor_demand_month_3 : Number_of_Employees_for_each_month_3 + 0.18000000000000002*Training_Decision_1 + 0.18000000000000002*Training_Decision_2 - Training_Decision_3 >= 115.0 	 770.99301 		 -0.0
Labor_demand_month_4 : Number_of_Employees_for_each_month_4 + 0.18000000000000002*Training_Decision_1 + 0.1

In [118]:
output=[]
for i in month:
    var_output=[]
    var_output.append(Trained[i].varValue)
    var_output.append(Layoff[i].varValue)
    var_output.append(Num[i].varValue)
  
    output.append(var_output)


#print(output)    
col_names=['Training Decision','Layoff','number of employees']
month_names=['Month 1', 'Month 2', 'Month 3', 'Month 4', 'Month 5', 'Month 6']
output_df = pd.DataFrame(output,index=month_names, columns=col_names)

print("Total cost=", value(prob2.objective))
output_df## Format PuLP Output

Total cost= 2526979.8546599997


,Training Decision,Layoff,number of employees
Month 1,13.364683,6.581766,145.00000
Month 2,40.823877,4.211484,138.41823
Month 3,28.960691,2.552875,134.20675
Month 4,21.620741,1.344194,131.65388
Month 5,9.168280,0.818570,130.30968
Month 6,NaN,0.777642,129.49111


In [119]:
output=[]
constraint_name=[]
for name, c in list(prob2.constraints.items()):
    var_output = []
    # print(name)
    constraint_name.append(name)
    var_output.append(str(c))
    var_output.append(c.pi)
    var_output.append(c.slack)
    output.append(var_output)
# print(constraint_name)    
col_names = ["Constraints","Shadow Price","Slack"]
# output_df = pd.DataFrame(output, index=constraint_name, columns=col_names)
output_df = pd.DataFrame(output, index = constraint_name, columns = col_names)
output_df

,Constraints,Shadow Price,Slack
_C1,Number_of_Employees_for_each_month_1 = 145,8440.13690,-0.000000e+00
Labor_demand_month_1,Number_of_Employees_for_each_month_1 - Trainin...,0.00000,-3.163532e+01
Labor_demand_month_6,Number_of_Employees_for_each_month_6 + 0.18000...,6089.65230,-0.000000e+00
Labor_demand_month_2,Number_of_Employees_for_each_month_2 + 0.18000...,471.46017,-0.000000e+00
Labor_demand_month_3,Number_of_Employees_for_each_month_3 + 0.18000...,770.99301,-0.000000e+00
Labor_demand_month_4,Number_of_Employees_for_each_month_4 + 0.18000...,932.92113,-0.000000e+00
Labor_demand_month_5,Number_of_Employees_for_each_month_5 + 0.18000...,980.62004,-0.000000e+00
Inflow_outflow_balance_1,-Layoff_Decision_1 + Number_of_Employees_for_e...,-5726.45990,-0.000000e+00
Inflow_outflow_balance_2,-Layoff_Decision_2 + Number_of_Employees_for_e...,-3366.23170,-0.000000e+00
Inflow_outflow_balance_3,-Layoff_Decision_3 + Number_of_Employees_for_e...,-1197.07860,-0.000000e+00


In [120]:
from pulp import *
import pandas as pd

In [121]:
prob2 = LpProblem('Prob2', LpMinimize)
month = [1, 2, 3, 4, 5, 6]

Initial = 145
Demand_per = [100,100,115,150,140,125] 

# Create a dictionary for Demand_per_mon
Demand_per_mon= makeDict([month], Demand_per) 
print(Demand_per_mon)

{1: 100, 2: 100, 3: 115, 4: 150, 5: 140, 6: 125}


In [122]:
Trained = LpVariable.dicts('Training Decision', month,lowBound = 0, cat = 'Continuous')
Layoff = LpVariable.dicts('Layoff Decision', month, lowBound = 0, cat = 'Continuous')
Num =  LpVariable.dicts('Number of Employees for each month', month, lowBound = 0, cat = 'Continuous')
print(Trained)
print(Layoff)
print(Num)

{1: Training_Decision_1, 2: Training_Decision_2, 3: Training_Decision_3, 4: Training_Decision_4, 5: Training_Decision_5, 6: Training_Decision_6}
{1: Layoff_Decision_1, 2: Layoff_Decision_2, 3: Layoff_Decision_3, 4: Layoff_Decision_4, 5: Layoff_Decision_5, 6: Layoff_Decision_6}
{1: Number_of_Employees_for_each_month_1, 2: Number_of_Employees_for_each_month_2, 3: Number_of_Employees_for_each_month_3, 4: Number_of_Employees_for_each_month_4, 5: Number_of_Employees_for_each_month_5, 6: Number_of_Employees_for_each_month_6}


In [123]:
prob2 += lpSum([(6-i)*3300*0.9*Trained[i] for i in month[0:5]])\
      + 3000*Num[1]\
      + lpSum([3000*(Num[i] - sum(0.9*Trained[j] for j in month[0:i-1])) for i in month[1:6]])
prob2

Prob2:
MINIMIZE
3000*Number_of_Employees_for_each_month_1 + 3000*Number_of_Employees_for_each_month_2 + 3000*Number_of_Employees_for_each_month_3 + 3000*Number_of_Employees_for_each_month_4 + 3000*Number_of_Employees_for_each_month_5 + 3000*Number_of_Employees_for_each_month_6 + 1350.0*Training_Decision_1 + 1080.0*Training_Decision_2 + 810.0*Training_Decision_3 + 540.0*Training_Decision_4 + 270.0*Training_Decision_5 + 0.0
VARIABLES
Number_of_Employees_for_each_month_1 Continuous
Number_of_Employees_for_each_month_2 Continuous
Number_of_Employees_for_each_month_3 Continuous
Number_of_Employees_for_each_month_4 Continuous
Number_of_Employees_for_each_month_5 Continuous
Number_of_Employees_for_each_month_6 Continuous
Training_Decision_1 Continuous
Training_Decision_2 Continuous
Training_Decision_3 Continuous
Training_Decision_4 Continuous
Training_Decision_5 Continuous

In [124]:
prob2 += Num[1] == 145
prob2 += Num[1] - Trained[1] >= 100,'Labor demand month 1'
prob2 += Num[6] + 0.2*0.9*(sum(Trained[j] for j in month[0:5])) >= Demand_per_mon[6],'Labor demand month 6'
for i in month[1:5]:
    prob2 += Num[i] - Trained[i] + 0.2*0.9*(sum(Trained[j] for j in month[0:i-1])) >= Demand_per_mon[i],'Labor demand month %s'%i

# Next month’s total employment = reduce the layoff amount 
for i in month[0:5]:
    prob2 += Num[i] - Layoff[i] == Num[i+1],'Inflow-outflow balance %s'%i
    
# Each month‘s layoff = 5% of (total employees - trained programmers - programmers on training)
prob2 += Layoff[1] == 0.05*(Num[1] - Trained[1]),'Layoff month 1'
for i in month[1:5]:
     prob2 += Layoff[i] == 0.05*(Num[i] - sum(Trained[j] for j in month[0:i])),'Layoff month %s'%i
prob2 += Layoff[6] == 0.05*(Num[6] - Trained[1] - Trained[2] - Trained[3] - Trained[4] - Trained[5]),'Layoff month 6'

    
# Each month’s # of trained programmers <= # of untrained programmers
for i in month[1:5]:
      prob2 += Trained[i] <= Num[i] - sum(Trained[j] for j in month[0:i-1]),'Train limit month %s'%i
prob2 += Trained[1] <= Num[1],'Train limit month 1'

In [125]:
prob2.solve()
print("Status:",LpStatus[prob2.status])

Status: Optimal


In [126]:
for i in prob2.variables():
    print(i.name, "=", i.varValue,"\tReduced Cost =", i.dj)
print("The minimize total operating cost for 6 months is ", value(prob2.objective))

Layoff_Decision_1 = 6.2163947 	Reduced Cost = 0.0
Layoff_Decision_2 = 3.7803458 	Reduced Cost = 0.0
Layoff_Decision_3 = 2.0225753 	Reduced Cost = 0.0
Layoff_Decision_4 = 1.9214465 	Reduced Cost = -9.094947e-13
Layoff_Decision_5 = 1.8253742 	Reduced Cost = -4.5474735e-13
Layoff_Decision_6 = 1.7341055 	Reduced Cost = 0.0
Number_of_Employees_for_each_month_1 = 145.0 	Reduced Cost = 1.1368684e-13
Number_of_Employees_for_each_month_2 = 138.78361 	Reduced Cost = -7.1054274e-14
Number_of_Employees_for_each_month_3 = 135.00326 	Reduced Cost = 7.1054274e-14
Number_of_Employees_for_each_month_4 = 132.98068 	Reduced Cost = -8.5265128e-13
Number_of_Employees_for_each_month_5 = 131.05924 	Reduced Cost = 5.6843419e-14
Number_of_Employees_for_each_month_6 = 129.23386 	Reduced Cost = 9.094947e-13
Training_Decision_1 = 20.672106 	Reduced Cost = -4.5474735e-13
Training_Decision_2 = 42.504584 	Reduced Cost = 1.1368684e-13
Training_Decision_3 = 31.375064 	Reduced Cost = 7.3896445e-13
Training_Decision_4 =

In [127]:
print("\nSensitivity Analysis\nConstraint\t\t\t\t\t\t\tShadow Price\t\tSlack")
for name,c in list(prob2.constraints.items()):
    print(name,":",c,"\t",c.pi,"\t\t",c.slack)


Sensitivity Analysis
Constraint							Shadow Price		Slack
_C1 : Number_of_Employees_for_each_month_1 = 145 	 6482.1193 		 -0.0
Labor_demand_month_1 : Number_of_Employees_for_each_month_1 - Training_Decision_1 >= 100 	 0.0 		 -24.327889999999996
Labor_demand_month_6 : Number_of_Employees_for_each_month_6 + 0.18000000000000002*Training_Decision_1 + 0.18000000000000002*Training_Decision_2 + 0.18000000000000002*Training_Decision_3 + 0.18000000000000002*Training_Decision_4 + 0.18000000000000002*Training_Decision_5 >= 125.0 	 0.0 		 -21.253179999999986
Labor_demand_month_2 : Number_of_Employees_for_each_month_2 + 0.18000000000000002*Training_Decision_1 - Training_Decision_2 >= 100.0 	 384.12664 		 -0.0
Labor_demand_month_3 : Number_of_Employees_for_each_month_3 + 0.18000000000000002*Training_Decision_1 + 0.18000000000000002*Training_Decision_2 - Training_Decision_3 >= 115.0 	 601.15618 		 -0.0
Labor_demand_month_4 : Number_of_Employees_for_each_month_4 + 0.18000000000000002*Training_Decisio

In [128]:
output=[]
for i in month:
    var_output=[]
    var_output.append(Trained[i].varValue)
    var_output.append(Layoff[i].varValue)
    var_output.append(Num[i].varValue)

    output.append(var_output)


#print(output)    
col_names=['Training','Layoff','number of employees']
month_names=['Month 1', 'Month 2', 'Month 3', 'Month 4', 'Month 5', 'Month 6']
output_df = pd.DataFrame(output,index=month_names, columns=col_names)

print("Total cost=", value(prob2.objective))
output_df## Format PuLP Output

Total cost= 2535408.0456600003


,Training,Layoff,number of employees
Month 1,20.672106,6.216395,145.00000
Month 2,42.504584,3.780346,138.78361
Month 3,31.375064,2.022575,135.00326
Month 4,0.000000,1.921447,132.98068
Month 5,0.000000,1.825374,131.05924
Month 6,NaN,1.734106,129.23386


In [129]:
output=[]
constraint_name=[]
for name, c in list(prob2.constraints.items()):
    var_output = []
    # print(name)
    constraint_name.append(name)
    var_output.append(str(c))
    var_output.append(c.pi)
    var_output.append(c.slack)
    output.append(var_output)
# print(constraint_name)    
col_names = ["Constraints","Shadow Price","Slack"]
# output_df = pd.DataFrame(output, index=constraint_name, columns=col_names)
output_df = pd.DataFrame(output, index = constraint_name, columns = col_names)
output_df

,Constraints,Shadow Price,Slack
_C1,Number_of_Employees_for_each_month_1 = 145,6482.11930,-0.000000e+00
Labor_demand_month_1,Number_of_Employees_for_each_month_1 - Trainin...,0.00000,-2.432789e+01
Labor_demand_month_6,Number_of_Employees_for_each_month_6 + 0.18000...,0.00000,-2.125318e+01
Labor_demand_month_2,Number_of_Employees_for_each_month_2 + 0.18000...,384.12664,-0.000000e+00
Labor_demand_month_3,Number_of_Employees_for_each_month_3 + 0.18000...,601.15618,-0.000000e+00
Labor_demand_month_4,Number_of_Employees_for_each_month_4 + 0.18000...,9919.70080,-0.000000e+00
Labor_demand_month_5,Number_of_Employees_for_each_month_5 + 0.18000...,0.00000,-8.078550e+00
Inflow_outflow_balance_1,-Layoff_Decision_1 + Number_of_Employees_for_e...,-3665.38880,-0.000000e+00
Inflow_outflow_balance_2,-Layoff_Decision_2 + Number_of_Employees_for_e...,-1104.75310,-2.842171e-14
Inflow_outflow_balance_3,-Layoff_Decision_3 + Number_of_Employees_for_e...,1362.20080,-0.000000e+00
